## Load Dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
# libgomp issue, must import n2 before torch
from n2 import HnswIndex

In [4]:
import sys

sys.path.insert(0, '..')

In [5]:
import os
home_dir = os.getenv('HOME')

https://dbs.uni-leipzig.de/research/projects/object_matching/benchmark_datasets_for_entity_resolution

https://www.informatik.uni-leipzig.de/~saeedi/NCVoters_Readme.txt

```
5 party:
--------- 
5 sources. 
Each source 1000,000 entities.
There is one file per source, so totally 5 files
****************************************************
****************************************************
****************************************************
Fields:
---------- 
recId: entites with the same recId refer to the same entity.
givenname: 
surname: 
post code: 
suburb:
```

In [6]:
import glob
import csv
from tqdm.auto import tqdm

current_row_id = 0
row_dict = {}
rows_total = 5000000
cluster_attr = 'recid'

with tqdm(total=rows_total) as pbar:
    for filename in glob.glob(f'{home_dir}/Downloads/5Party-ocp20/*.csv'):
        with open(filename) as f:
            for row in csv.DictReader(f):
                row['id'] = current_row_id
                row[cluster_attr] = int(row[cluster_attr])  # convert cluster_attr to int
                row_dict[current_row_id] = row
                current_row_id += 1
                pbar.update(1)

In [7]:
row_dict[0]

{'recid': 7852009,
 'givenname': 'kadelyn',
 'surname': 'gragnani',
 'suburb': 'waxhaw',
 'postcode': '28|73',
 'id': 0}

## Preprocess

In [8]:
attr_list = ['givenname', 'surname', 'suburb', 'postcode']

In [9]:
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()[:30]

for row in tqdm(row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [10]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [11]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [12]:
attr_info_dict = {
    'givenname': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'surname': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'suburb': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'postcode': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    }
}

In [13]:
from entity_embed import build_row_numericalizer

row_numericalizer = build_row_numericalizer(attr_info_dict, row_dict=row_dict)
row_numericalizer.attr_info_dict

15:47:13 INFO:For attr='givenname', computing actual alphabet and max_str_len
15:47:17 INFO:For attr='givenname', using actual_max_str_len=16
15:47:17 INFO:For attr='surname', computing actual alphabet and max_str_len
15:47:25 INFO:actual_max_str_len=21 must be pair to enable NN pooling. Updating to 22
15:47:25 INFO:For attr='surname', using actual_max_str_len=22
15:47:25 INFO:For attr='suburb', computing actual alphabet and max_str_len
15:47:32 INFO:actual_max_str_len=21 must be pair to enable NN pooling. Updating to 22
15:47:32 INFO:For attr='suburb', using actual_max_str_len=22
15:47:32 INFO:For attr='postcode', computing actual alphabet and max_str_len
15:47:36 INFO:actual_max_str_len=9 must be pair to enable NN pooling. Updating to 10
15:47:36 INFO:For attr='postcode', using actual_max_str_len=10


{'givenname': NumericalizeInfo(field_type=<FieldType.STRING: 'string'>, tokenizer=None, alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=16, vocab=None),
 'surname': NumericalizeInfo(field_type=<FieldType.STRING: 'string'>, tokenizer=None, alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=22, vocab=None),
 'suburb': NumericalizeInfo(field_type=<FieldType.STRI

In [14]:
from entity_embed import DeduplicationDataModule

train_cluster_len = 1000
valid_cluster_len = 1000
test_cluster_len = 200_000
datamodule = DeduplicationDataModule(
    row_dict=row_dict,
    cluster_attr=cluster_attr,
    row_numericalizer=row_numericalizer,
    batch_size=50,
    row_batch_size=16,
    train_cluster_len=train_cluster_len,
    valid_cluster_len=valid_cluster_len,
    test_cluster_len=test_cluster_len,
    only_plural_clusters=True,
    random_seed=random_seed
)

## Training

In [15]:
from entity_embed import EntityEmbed

ann_k = 10
model = EntityEmbed(
    datamodule,
    ann_k=ann_k,
)

In [16]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 50
early_stop_callback = EarlyStopping(
   monitor='valid_recall_at_0.5',
   min_delta=0.00,
   patience=10,
   verbose=True,
   mode='max'
)
tb_log_dir = 'tb_logs'
tb_name = 'voters'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name)
)

15:47:36 INFO:GPU available: True, used: True
15:47:36 INFO:TPU available: None, using: 0 TPU cores
15:47:36 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [17]:
trainer.fit(model, datamodule)

15:47:55 WARNING:train_len + valid_len + test_len=202000 is less than len(all_cluster_id_set)=500000
15:47:58 INFO:Train pair count: 6679
15:47:58 INFO:Valid pair count: 6645
15:47:58 INFO:Test pair count: 1331364
15:48:01 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 2.5 M 
1 | losser      | SupConLoss | 0     
-------------------------------------------
2.5 M     Trainable params
0         Non-trainable params
2.5 M     Total params


1

In [18]:
model.blocker_net.get_signature_weights()

{'givenname': 0.28603535890579224,
 'surname': 0.29768940806388855,
 'suburb': 0.22911803424358368,
 'postcode': 0.18715722858905792}

In [19]:
import gc
del trainer
gc.collect()

327

## Testing manually 

In [20]:
datamodule.setup(stage='test')

15:49:05 WARNING:train_len + valid_len + test_len=202000 is less than len(all_cluster_id_set)=500000
15:49:07 INFO:Train pair count: 6679
15:49:07 INFO:Valid pair count: 6645
15:49:07 INFO:Test pair count: 1331364


In [21]:
test_row_dict = datamodule.test_row_dict
test_vector_dict = model.predict(
    row_dict=test_row_dict,
    batch_size=16
)

In [22]:
embedding_size = model.blocker_net.embedding_size
test_true_pair_set = datamodule.test_true_pair_set

In [23]:
import gc
del model
del datamodule
gc.collect()

3149

In [24]:
assert len(test_vector_dict) == len(test_row_dict)

In [25]:
%%time

from entity_embed import ANNEntityIndex

ann_index = ANNEntityIndex(embedding_size=embedding_size)
ann_index.insert_vector_dict(test_vector_dict)
ann_index.build()

CPU times: user 13min 51s, sys: 2.04 s, total: 13min 53s
Wall time: 1min 20s


In [26]:
%%time

sim_threshold = 0.5
found_pair_set = ann_index.search_pairs(
    k=ann_k,
    sim_threshold=sim_threshold
)

CPU times: user 19min 22s, sys: 632 ms, total: 19min 23s
Wall time: 1min 44s


In [27]:
from entity_embed.evaluation import pair_entity_ratio

pair_entity_ratio(len(found_pair_set), len(test_row_dict))

6.122464070949708

In [28]:
from entity_embed.evaluation import precision_and_recall

precision_and_recall(found_pair_set, test_true_pair_set)

(0.27204511441381735, 0.9998467736847324)

In [29]:
false_positives = list(found_pair_set - test_true_pair_set)
len(false_positives)

3561999

In [30]:
false_negatives = list(test_true_pair_set - found_pair_set)
len(false_negatives)

204

In [31]:
cos_similarity = lambda a, b: np.dot(a, b)

In [32]:
for (id_left, id_right) in false_negatives[:10]:
    display(
        (
            cos_similarity(test_vector_dict[id_left], test_vector_dict[id_right]),
            row_dict[id_left], row_dict[id_right]
        )
    )

(0.7033602,
 {'recid': 6236158,
  'givenname': 'xelby',
  'surname': 'stmith',
  'suburb': 'charlotte',
  'postcode': '28208',
  'id': 2784},
 {'recid': 6236158,
  'givenname': 'shelby',
  'surname': 'smith',
  'suburb': 'charlotte',
  'postcode': '28208',
  'id': 2443031})

(0.750351,
 {'recid': 6045119,
  'givenname': 'dorotby',
  'surname': 'piekut',
  'suburb': 'hampstdad',
  'postcode': '28443',
  'id': 3005275},
 {'recid': 6045119,
  'givenname': 'dorothy',
  'surname': 'piekut',
  'suburb': 'haiiipstead',
  'postcode': '28447',
  'id': 4031596})

(0.76211625,
 {'recid': 7143003,
  'givenname': 'clarius',
  'surname': 'johnson',
  'suburb': 'elizabeth kity',
  'postcode': '27909',
  'id': 26513},
 {'recid': 7143003,
  'givenname': 'darius',
  'surname': 'johnxon',
  'suburb': 'elizabeth city',
  'postcode': '2790q',
  'id': 2026652})

(0.7812703,
 {'recid': 4881607,
  'givenname': 'james',
  'surname': 'loklear',
  'suburb': 'lumberton',
  'postcode': '2836o',
  'id': 2008239},
 {'recid': 4881607,
  'givenname': 'james',
  'surname': 'iocklear',
  'suburb': 'lumberton',
  'postcode': '28369',
  'id': 3008320})

(0.6819374,
 {'recid': 6218759,
  'givenname': 'thomas',
  'surname': 'kay',
  'suburb': 'wilmington',
  'postcode': '26411',
  'id': 1048595},
 {'recid': 6218759,
  'givenname': 'thoiiias',
  'surname': 'gau',
  'suburb': 'wilmington',
  'postcode': '28411',
  'id': 2048348})

(0.716912,
 {'recid': 225502,
  'givenname': 'barbara',
  'surname': 'beard',
  'suburb': 'lenoir',
  'postcode': '28645',
  'id': 346539},
 {'recid': 225502,
  'givenname': 'barbara',
  'surname': 'loeard',
  'suburb': 'lenoiah',
  'postcode': '28645',
  'id': 1019069})

(0.883077,
 {'recid': 120716,
  'givenname': 'charles',
  'surname': 'jones',
  'suburb': 'shelby',
  'postcode': '28150',
  'id': 1370588},
 {'recid': 120716,
  'givenname': 'charles',
  'surname': 'jonws',
  'suburb': 'shelby',
  'postcode': '28152',
  'id': 4060876})

(0.8352002,
 {'recid': 7839955,
  'givenname': 'danvs',
  'surname': 'johnson',
  'suburb': 'concord',
  'postcode': '28026',
  'id': 34454},
 {'recid': 7839955,
  'givenname': 'danuj',
  'surname': 'johhson',
  'suburb': 'concord',
  'postcode': '28027',
  'id': 2034465})

(0.6355559,
 {'recid': 943532,
  'givenname': 'tere5a',
  'surname': 'iiiurray',
  'suburb': 'concord',
  'postcode': '28027',
  'id': 1037384},
 {'recid': 943532,
  'givenname': 'teresa',
  'surname': 'murray',
  'suburb': 'concord',
  'postcode': '28027',
  'id': 3961448})

(0.6355559,
 {'recid': 943532,
  'givenname': 'teresa',
  'surname': 'murray',
  'suburb': 'concord',
  'postcode': '28027',
  'id': 961611},
 {'recid': 943532,
  'givenname': 'tere5a',
  'surname': 'iiiurray',
  'suburb': 'concord',
  'postcode': '28027',
  'id': 1037384})